<a href="https://colab.research.google.com/github/tararajagopalan/WebsiteQueryingProject/blob/main/projectdatatable_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#INSTALLING PYMILVUS

!pip install -U pymilvus
!pip install --upgrade pymilvus
!pip install "pymilvus[model]"

In [ ]:
#SETTING UP VECTOR DATA BASE: creating a database on the mounted google drive
#sets up data base in the project_work.db file on google drive
from pymilvus import MilvusClient

client = MilvusClient("/content/drive/MyDrive/project_work.db")

In [ ]:
#checks to see if client has collection table project_collection; if it alr does, then drop it!
if client.has_collection(collection_name="project_collection"):
  client.drop_collection(collection_name="project_collection")

In [ ]:
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType

#below are all of the fields that we want in the project_collection table!
#FileName is probably going to be the path of the file or name of the file?
#FileExtension is the file type; ex .pdf is pdf, .mp4 is video, .jgp is photo

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True), #automatically fills in the id field
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=2048),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=768),
    FieldSchema(name="FilePath", dtype=DataType.VARCHAR, max_length=80),
    FieldSchema(name="FileExtension", dtype=DataType.VARCHAR, max_length=80)
]

schema = CollectionSchema(fields=fields)

In [ ]:
# Create the collection called project_collection with the schema created in the previous code block
client.create_collection(
    collection_name="project_collection",
    schema=schema,
)

In [ ]:
index_params = client.prepare_index_params()

#adding an index called 'vector_index' of type 'IVF_FLAT' for the field 'embedding'
#instead of cosine similarity its using L2 similarity
index_params.add_index(
    field_name="embedding", # Name of the vector field to be indexed
    index_type="IVF_FLAT", # Type of the index to create
    index_name="vector_index", # Name of the index to create
    metric_type="L2", # Metric type used to measure similarity
    params={
        "nlist": 128, # Number of clusters for the index
    } # Index building params
)

client.create_index(
        collection_name="project_collection",
        index_params=index_params,
    )

client.flush("project_collection")